# Check Python Version

In [45]:
 import sys
 print(sys.executable)
 print(sys.version)
 print(sys.version_info)

c:\python38\python.exe
3.8.3 (tags/v3.8.3:6f8c832, May 13 2020, 22:37:02) [MSC v.1924 64 bit (AMD64)]
sys.version_info(major=3, minor=8, micro=3, releaselevel='final', serial=0)


# Load libraries

In [2]:
import numpy as np
import pandas as pd
import os

# Specify git executable file for GitPython in Jupyter Notebook (In IDE, it can still work without this line.)
os.environ["GIT_PYTHON_GIT_EXECUTABLE"] = "C:\Program Files\Git\cmd\git.exe"

import git
from git import RemoteProgress

from git import Repo
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

# Clone repo from GitHub

Link: https://git-scm.com/docs/git-clone <br>
<b>Note:</b> In case too-long file path issue occurs in Windows, set <code>git config --system core.longpaths true</code>

In [3]:
class Progress(RemoteProgress):
    def update(self, op_code, cur_count, max_count=None, message=''):
        print(self._cur_line)

In [4]:
remote_link = "https://github.com/jenkinsci/jenkins"
local_link = "repo/jenkins"
# Uncomment to clone
Repo.clone_from(remote_link, local_link, progress=Progress())

remote: Counting objects: 100% (1/1)
remote: Counting objects: 100% (1/1), done.
Receiving objects:   0% (1/372018)
Receiving objects:   0% (1185/372018), 452.00 KiB | 410.00 KiB/s
Receiving objects:   0% (3326/372018), 780.00 KiB | 458.00 KiB/s
Receiving objects:   1% (3721/372018), 780.00 KiB | 458.00 KiB/s
Receiving objects:   1% (6667/372018), 1.56 MiB | 552.00 KiB/s
Receiving objects:   2% (7441/372018), 1.56 MiB | 552.00 KiB/s
Receiving objects:   2% (11047/372018), 2.65 MiB | 664.00 KiB/s
Receiving objects:   3% (11161/372018), 2.65 MiB | 664.00 KiB/s
Receiving objects:   4% (14881/372018), 3.27 MiB | 730.00 KiB/s
Receiving objects:   4% (16600/372018), 3.27 MiB | 730.00 KiB/s
Receiving objects:   5% (18601/372018), 3.85 MiB | 774.00 KiB/s
Receiving objects:   5% (20675/372018), 4.39 MiB | 841.00 KiB/s
Receiving objects:   6% (22322/372018), 4.80 MiB | 889.00 KiB/s
Receiving objects:   7% (26042/372018), 5.39 MiB | 963.00 KiB/s
Receiving objects:   7% (26428/372018), 5.39 MiB | 

Receiving objects:  91% (338537/372018), 111.04 MiB | 3.95 MiB/s
Receiving objects:  92% (342257/372018), 111.04 MiB | 3.95 MiB/s
Receiving objects:  93% (345977/372018), 112.89 MiB | 3.84 MiB/s
Receiving objects:  94% (349697/372018), 112.89 MiB | 3.84 MiB/s
Receiving objects:  94% (350037/372018), 112.89 MiB | 3.84 MiB/s
Receiving objects:  95% (353418/372018), 112.89 MiB | 3.84 MiB/s
Receiving objects:  96% (357138/372018), 112.89 MiB | 3.84 MiB/s
Receiving objects:  97% (360858/372018), 114.95 MiB | 3.74 MiB/s
Receiving objects:  98% (364578/372018), 114.95 MiB | 3.74 MiB/s
Receiving objects:  99% (368298/372018), 116.71 MiB | 3.54 MiB/s
Receiving objects:  99% (369719/372018), 116.71 MiB | 3.54 MiB/s
Receiving objects: 100% (372018/372018), 116.71 MiB | 3.54 MiB/s
Receiving objects: 100% (372018/372018), 117.63 MiB | 3.01 MiB/s, done.
Resolving deltas:   0% (0/244302)
Resolving deltas:   1% (2452/244302)
Resolving deltas:   2% (5109/244302)
Resolving deltas:   3% (7391/244302)
Res

<git.repo.base.Repo 'D:\\University Work\\SSE\\Assignment 2\\repo\\jenkins\\.git'>

In [5]:
repo = Repo(local_link)
fixing_commit = "ccc374a7176d7704941fb494589790b7673efe2e"
affected_file = "core/src/main/java/jenkins/model/Jenkins.java"

# Show info of fixing commit

In [154]:
show_data = repo.git.show("-s", fixing_commit).splitlines()
print("Fixing Commit:")
for line in show_data:
    print(line)

Fixing Commit:
commit ccc374a7176d7704941fb494589790b7673efe2e
Author: Jesse Glick <jglick@cloudbees.com>
Date:   Wed Dec 13 12:59:04 2017 -0500

    [SECURITY-667] Ensure all tasks have completed before we attain COMPLETED.


# Find the line that fixed this vulnerability
<b>Note:</b> May not work if the line contains the @@ - characters, however because of unified=0 this should be fine. This will only grab the latest line.

In [79]:
diff_data = repo.git.diff("--unified=0", fixing_commit + "^", fixing_commit).splitlines()
actual_line = -1
line_string = ""
for line in diff_data:
    # print(line)
    if(line.find("@@ -") != -1):
        line_string = line[4: line.find(" +")]

print("Line with vulnerabiltiy fix is: " + line_string)

Line with vulnerabiltiy fix is: 3115


# Identify the previous commit that changed the vulnerable line

<b>Note:</b> This will use the "-wC" paramter to ignore changes in the file name or movement in the directory.

In [83]:
blame_data = repo.git.blame("-L"+line_string+",+1","-wC",fixing_commit + "^", "--", affected_file).splitlines()
vulnerable_commit = ""
for line in blame_data:
    # print(line)
    vulnerable_commit = line[0: line.find(" ")]

print("The vulnerablility contributing commit was: "+vulnerable_commit)

The vulnerablility contributing commit was: fd13eee6d53


# Get information about the commit

## Message and Title

In [59]:
show_data = repo.git.show(vulnerable_commit).splitlines()
for line in show_data:
    print(line)

commit fd13eee6d53af7b6f0cf94bd3e4d290dfa2286cb
Author: kohsuke <kohsuke@71c3de6d-444a-0410-be80-ed276b4c234a>
Date:   Mon Nov 23 19:33:33 2009 +0000

    Merged revisions 23666-23667,23670,23711,23782-23785,23788,23809-23810,23817,23859,23870,23879,23899-23904,23906-23907,23909,23913 via svnmerge from
    https://www.dev.java.net/svn/hudson/branches/managed-startup
    
    ........
      r23666 | kohsuke | 2009-11-11 15:56:46 -0800 (Wed, 11 Nov 2009) | 1 line
    
      Reworked the initialization logic
    ........
      r23667 | kohsuke | 2009-11-11 16:01:56 -0800 (Wed, 11 Nov 2009) | 1 line
    
      fixed a failing test
    ........
      r23670 | kohsuke | 2009-11-11 16:42:32 -0800 (Wed, 11 Nov 2009) | 1 line
    
      use a non-snapshot
    ........
      r23711 | kohsuke | 2009-11-13 14:27:28 -0800 (Fri, 13 Nov 2009) | 1 line
    
      Session -> Reactor
    ........
      r23782 | kohsuke | 2009-11-17 10:35:14 -0800 (Tue, 17 Nov 2009) | 1 line
    
      Poorman's clone of

## Files and Directories Affected
Also shows total lines of coded added or deleted including comments and blank lines.

In [77]:
file_data = repo.git.show("--format=","--stat","--dirstat",vulnerable_commit).splitlines()
insertions = -1
deletions = -1
print("Affected files and directories:")
for line in file_data:
    print(line)
    placeholder_line = line.find("files changed, ")
    if(placeholder_line != -1):
        insertions = line[int(placeholder_line)+15: line.find(" insertions(+)")]
        deletions = line[line.find(" insertions(+), ")+16: line.find(" deletions(-)")]

print("Insertions: " + insertions)
print("Deletions: " + deletions)

Affected files and directories:
 core/pom.xml                                       |   5 +
 core/src/main/java/hudson/BulkChange.java          |  12 +-
 .../main/java/hudson/ClassicPluginStrategy.java    |  40 +-
 core/src/main/java/hudson/PluginManager.java       | 184 +++++---
 core/src/main/java/hudson/PluginStrategy.java      |   3 +-
 core/src/main/java/hudson/PluginWrapper.java       |   3 +-
 core/src/main/java/hudson/init/InitMilestone.java  |  91 ++++
 .../main/java/hudson/init/InitReactorListener.java |  22 +
 core/src/main/java/hudson/init/InitStrategy.java   | 101 +++++
 core/src/main/java/hudson/init/Initializer.java    |  92 ++++
 .../main/java/hudson/init/InitializerFinder.java   | 183 ++++++++
 .../java/hudson/init/impl/GroovyInitScript.java    |  54 +++
 .../java/hudson/init/impl/InitialUserContent.java  |  48 +++
 core/src/main/java/hudson/init/package-info.java   |  47 +++
 .../java/hudson/logging/LogRecorderManager.java    |   6 +
 core/src/main/java/hudson/model/H

## Lines of code excluding comments and blank lines

In [84]:
# Re for regualr expression searching
import re
comment_additions = 0
comment_deletions = 0
blankline_additions = 0
blankline_deletions = 0
for line in show_data:
    if re.search('^[+]\s*[*]', line) or re.search('^[+]\s*[\/][*]', line) or re.search('^[+]\s*[\/][\/]', line):
        comment_additions = comment_additions + 1
    elif re.search('^[-]\s*[*]', line) or re.search('^[-]\s*[\/][*]', line) or re.search('^[-]\s*[\/][\/]', line):
        comment_deletions = comment_deletions + 1
    elif re.search('^[+]\s*$', line):
        blankline_additions = blankline_additions + 1
    elif re.search('^[-]\s*$', line):
        blankline_deletions = blankline_deletions + 1

# print("Comment additions: " + str(comment_additions))
# print("Comment deletions: " + str(comment_deletions))
# print("Blank line additions: " + str(blankline_additions))
# print("Blank line deletions: " + str(blankline_deletions))
# print("Total Ignored additions: " + str(blankline_additions + comment_additions))
# print("Total Ignored deletions: " + str(blankline_deletions + comment_deletions))
print("Total lines added excluding comments and blank lines: " + str(int(insertions) - (blankline_additions + comment_additions)))
print("Total lines deleted excluding comments and blank lines: " + str(int(deletions) - (blankline_deletions + comment_deletions)))

Total lines added excluding comments and blank lines: 650
Total lines deleted excluding comments and blank lines: 251


## Days between file modification in commit and total commits

In [118]:
import datetime

show_files_only = repo.git.show("--format=","--name-only",vulnerable_commit).splitlines()
# Time of this commit and convert
this_commit = repo.git.log("-1","--format=%ci",vulnerable_commit)
this_commit = datetime.datetime.strptime(this_commit,'%Y-%m-%d %H:%M:%S %z')
# print(this_commit)
for line in show_files_only:
    last_commit = 0
    # Get time of last commit, exception means the commit doesn't exist which means it came from the VCC
    try:
        last_commit = repo.git.log("-1","--skip","1","--format=%ci",vulnerable_commit,"--",line)
        last_commit = datetime.datetime.strptime(last_commit,'%Y-%m-%d %H:%M:%S %z')
    except Exception as e:
        # print(str(e))
        last_commit = this_commit
    total_commits = repo.git.log("--name-only","--format=",vulnerable_commit,"--",line).splitlines()
    total_commit_num = 0
    for commits in total_commits:
        if re.search(line,commits):
            total_commit_num = total_commit_num + 1
    print(line + " - " + str((this_commit - last_commit)) + " - Total Commits: " + str(total_commit_num))
    

core/pom.xml - 2 days, 20:39:15 - Total Commits: 881
core/src/main/java/hudson/BulkChange.java - 289 days, 23:28:09 - Total Commits: 3
core/src/main/java/hudson/ClassicPluginStrategy.java - 55 days, 23:34:40 - Total Commits: 22
core/src/main/java/hudson/PluginManager.java - 24 days, 3:04:44 - Total Commits: 38
core/src/main/java/hudson/PluginStrategy.java - 289 days, 23:28:09 - Total Commits: 5
core/src/main/java/hudson/PluginWrapper.java - 24 days, 22:05:47 - Total Commits: 38
core/src/main/java/hudson/init/InitMilestone.java - 0:00:00 - Total Commits: 1
core/src/main/java/hudson/init/InitReactorListener.java - 0:00:00 - Total Commits: 1
core/src/main/java/hudson/init/InitStrategy.java - 0:00:00 - Total Commits: 1
core/src/main/java/hudson/init/Initializer.java - 0:00:00 - Total Commits: 1
core/src/main/java/hudson/init/InitializerFinder.java - 0:00:00 - Total Commits: 1
core/src/main/java/hudson/init/impl/GroovyInitScript.java - 0:00:00 - Total Commits: 1
core/src/main/java/hudson/in

## Developers who have modified each file and their number of commits

In [152]:
print("Developers by file:")
developer_dict = {}

for line in show_files_only:
    print(line)
    developers = repo.git.shortlog("--summary",vulnerable_commit,"--",line).splitlines()
    for developer in developers:
        print(developer)
        # Get the number of commits and developer name and add them to the dictionary
        test = re.search("^\s+\d",developer)
        test2 = re.search("\s+[a-zA-Z]",developer)
        test3 = re.search("[a-zA-Z]+$",developer)
        if(test != None):
            commits = developer[test.end()-1 : test2.start()]
            dev = developer[test3.start() : test3.end()]
            if(dev in developer_dict):
                developer_dict[dev] = int(developer_dict[dev]) + int(commits)
            else:
                developer_dict[dev] = int(commits)

Developers by file:
core/pom.xml
     2	abayer
     1	dty
     1	dwdyer
     1	huybrechts
     7	jglick
   864	kohsuke
     1	mdonohue
     2	mindless
     1	redsolo
     1	stephenconnolly
core/src/main/java/hudson/BulkChange.java
     3	kohsuke
core/src/main/java/hudson/ClassicPluginStrategy.java
     1	huybrechts
     1	jbq
     1	jglick
    17	kohsuke
     2	mindless
core/src/main/java/hudson/PluginManager.java
     1	huybrechts
     1	jglick
    33	kohsuke
     2	mindless
     1	stephenconnolly
core/src/main/java/hudson/PluginStrategy.java
     1	huybrechts
     3	kohsuke
     1	mindless
core/src/main/java/hudson/PluginWrapper.java
     2	dty
     1	huybrechts
     1	jglick
    31	kohsuke
     1	mindless
     2	redsolo
core/src/main/java/hudson/init/InitMilestone.java
     1	kohsuke
core/src/main/java/hudson/init/InitReactorListener.java
     1	kohsuke
core/src/main/java/hudson/init/InitStrategy.java
     1	kohsuke
core/src/main/java/hudson/init/Initializer.java
     1	kohsuke
core

## Total number of commits by developers on affected files

In [153]:
print("Developers by commits on affected files:")
for x, y in developer_dict.items():
  print(x, y)

Developers by commits on affected files:
abayer 14
dty 7
dwdyer 1
huybrechts 17
jglick 15
kohsuke 1797
mdonohue 3
mindless 46
redsolo 6
stephenconnolly 19
jbq 3
Ballance 1
dvrzalik 1
fujibee 2
lifeless 1
shinodkm 1
sogabe 18
pgweiss 2
bwestrich 1
cactusman 3
elefevre 1
swiest 1
